In [45]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam


In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [47]:
df = pd.read_csv('data2.csv')

In [ ]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,27.920,28.325,27.920,28.185,7146296,SBUX
1,2013-02-11,28.260,28.260,27.930,28.070,5457354,SBUX
2,2013-02-12,28.000,28.275,27.975,28.130,8665592,SBUX
3,2013-02-13,28.230,28.230,27.750,27.915,7022056,SBUX
4,2013-02-14,27.765,27.905,27.675,27.775,8899188,SBUX


In [48]:
series = df['close'].values.reshape(-1, 1)
# series = df['close'].values


In [49]:
series.shape

(1259, 1)

In [50]:
scaler = StandardScaler()
scaler.fit(series[:len(series) // 2])
series = scaler.transform(series).flatten()

In [51]:
series.shape

(1259,)

In [52]:
T = 10
D = 1
X = []
Y = []

for t in range(len(series)-T):
  X.append(series[t: t+T])
  Y.append(series[t+T])

X = np.array(X).reshape(-1, T, 1)
Y = np.array(Y)


In [53]:
i = Input(shape=(T, 1))
#x = LSTM(3, return_sequences = True)(i)
x = LSTM(5, return_sequences = True)(i)
x = LSTM(7)(x)
x = Dense(1)(x)
model = Model(i, x)
model.compile(
    loss='mse', 
    optimizer=Adam(lr=0.1)
)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [54]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 5)             140       
                                                                 
 lstm_3 (LSTM)               (None, 7)                 364       
                                                                 
 dense_1 (Dense)             (None, 1)                 8         
                                                                 
Total params: 512
Trainable params: 512
Non-trainable params: 0
_________________________________________________________________


In [ ]:
N = len(X)
r = model.fit(
    X[:N//2], Y[:N//2], 
    epochs=80, 
    validation_data=(X[-N//2:], Y[-N//2:])
)

Epoch 1/80
20/20 [==============================] - 9s 135ms/step - loss: 0.2016 - val_loss: 0.2436
Epoch 2/80
20/20 [==============================] - 0s 21ms/step - loss: 0.0228 - val_loss: 0.0702
Epoch 3/80
20/20 [==============================] - 0s 23ms/step - loss: 0.0088 - val_loss: 0.0602
Epoch 4/80
20/20 [==============================] - 0s 21ms/step - loss: 0.0092 - val_loss: 0.0428
Epoch 5/80
20/20 [==============================] - 0s 25ms/step - loss: 0.0093 - val_loss: 0.0664
Epoch 6/80
20/20 [==============================] - 1s 28ms/step - loss: 0.0158 - val_loss: 0.1658
Epoch 7/80
20/20 [==============================] - 1s 34ms/step - loss: 0.0140 - val_loss: 0.1062
Epoch 8/80
20/20 [==============================] - 0s 19ms/step - loss: 0.0087 - val_loss: 0.0528
Epoch 9/80
14/20 [====================>.........] - ETA: 0s - loss: 0.0075

In [ ]:
from matplotlib.pyplot import figure

output = model.predict(X[N//2:])
output[:, 0]
plt.plot(Y[N//2:], label='Real Price')
plt.plot(output[:, 0], label='Predict')
plt.legend()
plt.show()


In [ ]:
np.sum((Y[N//2:] - output[:, 0]) * (Y[N//2:] - output[:, 0]))

In [ ]:
results = model.evaluate(X[:N//2], Y[:N//2] )
print(results)

In [ ]:
plt.plot(r.history['val_loss'])

In [ ]:
trade = []
for i in range(len(X)//2):
  if(X[len(X)//2 + i][9][0] < Y[len(X)//2 + i]):
    trade.append(1)
  else:
    trade.append(0)
    

In [ ]:
trade = np.array(trade)

In [ ]:
predict_trade = []
for i in range(len(X)//2):
  if(X[len(X)//2 + i][9][0] < output[:, 0][i]):
    predict_trade.append(1)
  else: 
    predict_trade.append(0)

In [ ]:
print(len(trade), len(predict_trade))

In [ ]:
count = 0
for i in range(len(trade)):
  if(trade[i] == predict_trade[i]):
    count += 1

In [ ]:
count

In [ ]:
327/624

In [ ]:
profit = 0

for i in range(len(trade)):
  if(predict_trade[i] == 1):
    profit += Y[len(X)//2 + i] - X[len(X)//2 + i][9][0]


In [ ]:
profit